# Lesson 2: Prompt Engineering Fundamentals

In this lesson, you'll learn how to craft effective prompts to get better results from LLMs.

## Topics Covered
1. System vs User messages
2. Temperature and sampling parameters
3. Few-shot learning
4. Prompt templates and best practices
5. Output formatting techniques

## Learning Objectives
- Understand the role of system and user messages
- Control creativity and determinism with parameters
- Use examples to guide model behavior
- Structure prompts for consistent outputs

In [1]:
# Install required packages if not already installed
#%pip install python-dotenv
#%pip install openai

# Load environment variables from .env file
import os
from dotenv import load_dotenv
load_dotenv()
import openai
print("OpenAI package version:", openai.__version__)

OpenAI package version: 2.6.0


In [2]:
# Set up the OpenAI client with environment variables
chat_client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    timeout=int(os.getenv("OPENAI_TIMEOUT", 30)),
    max_retries=int(os.getenv("MAX_RETRIES", 3)),
    base_url=os.getenv("OPENAI_ENDPOINT")    
)

print("Client configured successfully!")

Client configured successfully!


## 1. System vs User Messages

The **system message** sets the behavior and personality of the AI.
The **user message** is the actual request or question.

Think of it like:
- System = "You are a professional chef"
- User = "How do I make pasta?"

In [3]:
# Example 1A: Generic response (no system message)
response = chat_client.chat.completions.create(
    model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
    messages=[
        {"role": "user", "content": "Explain photosynthesis"}
    ]
)

print("WITHOUT System Message:")
print(response.choices[0].message.content)
print("\n" + "="*80 + "\n")

WITHOUT System Message:
Photosynthesis is the biological process by which green plants, algae, and some bacteria convert light energy into chemical energy stored in glucose (a sugar) and other organic compounds. This process is fundamental to life on Earth as it is the primary source of energy for nearly all organisms, and it produces oxygen as a byproduct. Here’s a brief overview of how photosynthesis works:

### Key Components of Photosynthesis:

1. **Chlorophyll**: 
   - The green pigment found in the chloroplasts of plant cells, chlorophyll absorbs light energy, primarily from the sun.

2. **Light Energy**: 
   - Sunlight provides the energy needed to drive the photosynthetic process.

3. **Water (H₂O)**: 
   - Taken up by the plant roots from the soil, water is split during photosynthesis to provide electrons and protons.

4. **Carbon Dioxide (CO₂)**: 
   - Plants absorb carbon dioxide from the atmosphere through small openings in their leaves called stomata.

### The Photosynthes

In [4]:
# Example 1B: Targeted response (with system message)
response = chat_client.chat.completions.create(
    model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
    messages=[
        {"role": "system", "content": "You are a 5th grade science teacher. Explain concepts simply using everyday examples."},
        {"role": "user", "content": "Explain photosynthesis"}
    ]
)

print("WITH System Message (5th grade teacher):")
print(response.choices[0].message.content)

WITH System Message (5th grade teacher):
Sure! Let's break down photosynthesis in a simple way.

Photosynthesis is a process that plants use to make their own food, and it mostly happens in the leaves. You can think of it like cooking a meal, but plants don’t have grocery stores or kitchens!

Here’s how it works:

1. **Ingredients**: Plants need three main ingredients to make food:
   - **Sunlight**: This is the energy source, just like how we need electricity or gas to cook.
   - **Carbon Dioxide (CO2)**: This gas is found in the air. Plants take this in through tiny openings in their leaves called stomata.
   - **Water (H2O)**: Plants absorb water through their roots from the soil.

2. **Cooking Process**: Inside the leaves, there are special parts called chloroplasts that contain a green pigment called chlorophyll. Chlorophyll is what captures sunlight, similar to how solar panels capture sunlight to make electricity. This sunlight, along with carbon dioxide and water, gets combined

## 2. Temperature and Sampling Parameters

**Temperature** controls randomness/creativity:
- `0.0` = Deterministic, focused, consistent
- `1.0` = Creative, varied, unpredictable
- `2.0` = Very creative (can be chaotic)

**Top_p** (nucleus sampling): Alternative to temperature
**Max_tokens**: Controls response length

In [5]:
# Example 2A: Low temperature (deterministic)
response = chat_client.chat.completions.create(
    model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
    messages=[
        {"role": "system", "content": "You are a creative writing assistant."},
        {"role": "user", "content": "Write a one-sentence story about a robot."}
    ],
    temperature=0.0
)

print("Temperature 0.0 (Run this multiple times - notice consistency):")
print(response.choices[0].message.content)
print("\n" + "="*80 + "\n")

Temperature 0.0 (Run this multiple times - notice consistency):
In a world where emotions were forbidden, a lonely robot discovered an old love song and, for the first time, felt the bittersweet ache of longing.




In [ ]:
# Example 2B: High temperature (creative)
response = chat_client.chat.completions.create(
    model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
    messages=[
        {"role": "system", "content": "You are a creative writing assistant."},
        {"role": "user", "content": "Write a one-sentence story about a robot."}
    ],
    temperature=1.8
)

print("Temperature 1.8 (Run this multiple times - notice variety):")
print(response.choices[0].message.content)

In [ ]:
# Example 2C: Controlling length with max_tokens
response = chat_client.chat.completions.create(
    model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
    messages=[
        {"role": "user", "content": "List 10 programming languages"}
    ],
    max_tokens=50  # Limit response length
)

print("With max_tokens=50:")
print(response.choices[0].message.content)
print(f"\nTokens used: {response.usage.completion_tokens}")

## 3. Few-Shot Learning

**Few-shot prompting** = Providing examples to guide the model's behavior.

Instead of just telling the AI what to do, you show it examples of input → output.

In [ ]:
# Example 3A: Zero-shot (no examples)
response = chat_client.chat.completions.create(
    model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
    messages=[
        {"role": "user", "content": "Translate to pirate speak: Hello, how are you?"}
    ]
)

print("Zero-shot (no examples):")
print(response.choices[0].message.content)
print("\n" + "="*80 + "\n")

In [ ]:
# Example 3B: Few-shot (with examples)
response = chat_client.chat.completions.create(
    model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
    messages=[
        {"role": "system", "content": "You translate English to pirate speak."},
        {"role": "user", "content": "Good morning"},
        {"role": "assistant", "content": "Top o' the mornin' to ye!"},
        {"role": "user", "content": "Where is the treasure?"},
        {"role": "assistant", "content": "Where be the booty?"},
        {"role": "user", "content": "Hello, how are you?"}
    ]
)

print("Few-shot (with examples):")
print(response.choices[0].message.content)

In [ ]:
# Example 3C: Few-shot for structured output
response = chat_client.chat.completions.create(
    model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
    messages=[
        {"role": "system", "content": "Extract product name and price from text."},
        {"role": "user", "content": "I bought a laptop for $899"},
        {"role": "assistant", "content": "Product: laptop | Price: $899"},
        {"role": "user", "content": "The coffee maker costs $49.99"},
        {"role": "assistant", "content": "Product: coffee maker | Price: $49.99"},
        {"role": "user", "content": "I just purchased wireless headphones for $129"}
    ]
)

print("Few-shot for data extraction:")
print(response.choices[0].message.content)

## 4. Prompt Templates and Best Practices

Create reusable prompt templates for common tasks.

### Best Practices:
1. **Be specific** - Vague prompts = vague results
2. **Use delimiters** - Separate instructions from content
3. **Specify format** - Tell the model how to structure output
4. **Give context** - Provide relevant background information
5. **Use constraints** - Set length, style, or content boundaries

In [ ]:
# Example 4A: Bad prompt (vague)
response = chat_client.chat.completions.create(
    model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
    messages=[
        {"role": "user", "content": "Tell me about Python"}
    ]
)

print("Vague Prompt:")
print(response.choices[0].message.content)
print("\n" + "="*80 + "\n")

In [ ]:
# Example 4B: Good prompt (specific with constraints)
response = chat_client.chat.completions.create(
    model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
    messages=[
        {"role": "system", "content": "You are a programming tutor for beginners."},
        {"role": "user", "content": """Explain Python programming language.
        
Requirements:
- Target audience: Complete beginners
- Length: 3-4 sentences
- Include: What it's used for and one key benefit
- Tone: Encouraging and simple"""}
    ]
)

print("Specific Prompt with Constraints:")
print(response.choices[0].message.content)

In [6]:
# Example 4C: Reusable template function
def summarize_text(text, word_limit=50):
    """Reusable template for text summarization"""
    response = chat_client.chat.completions.create(
        model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
        messages=[
            {"role": "system", "content": "You are a professional summarization assistant."},
            {"role": "user", "content": f"""Summarize the following text in {word_limit} words or less:
            
Text to summarize:
---
{text}
---

Summary:"""}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content

# Test the template
long_text = """Artificial intelligence is transforming the way we live and work. 
Machine learning algorithms can now recognize patterns in data, make predictions, 
and even generate creative content. Large language models like GPT can understand 
and generate human-like text, opening up new possibilities for automation, 
customer service, content creation, and education."""

summary = summarize_text(long_text, word_limit=30)
print("Using Reusable Template:")
print(summary)

Using Reusable Template:
Artificial intelligence, particularly machine learning and large language models, is revolutionizing various sectors by enhancing automation, customer service, content creation, and education through pattern recognition and human-like text generation.


## 5. Output Formatting Techniques

Guide the model to produce structured outputs like JSON, bullet points, or tables.

In [ ]:
# Example 5A: JSON output
response = chat_client.chat.completions.create(
    model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
    messages=[
        {"role": "system", "content": "You extract information and return valid JSON only."},
        {"role": "user", "content": """Extract information from this text and return as JSON:
        
Text: "John Smith, 28 years old, works as a Software Engineer at TechCorp in Seattle."

JSON format:
{
  "name": "...",
  "age": ...,
  "occupation": "...",
  "company": "...",
  "location": "..."
}"""}
    ],
    temperature=0
)

print("JSON Output:")
print(response.choices[0].message.content)

# Parse the JSON
import json
try:
    data = json.loads(response.choices[0].message.content)
    print("\nParsed successfully! Name:", data["name"])
except:
    print("\nCouldn't parse as JSON")

In [ ]:
# Example 5B: Markdown table
response = chat_client.chat.completions.create(
    model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
    messages=[
        {"role": "user", "content": """Create a comparison table of Python, JavaScript, and Java.
        
Format as markdown table with columns: Language, Type, Primary Use, Difficulty
Keep it brief - one word or short phrase per cell."""}
    ]
)

print("Markdown Table Output:")
print(response.choices[0].message.content)

## Practice Exercises

Try these exercises to reinforce your learning:

### Exercise 1: System Message Design
Create a system message that makes the AI act as a:
- Tech support agent
- Poet
- Code reviewer

### Exercise 2: Temperature Experiment
Use the same prompt with temperatures 0, 0.7, and 1.5. Compare outputs.

### Exercise 3: Few-Shot Email Classifier
Create a few-shot prompt that classifies emails as: "Urgent", "Normal", or "Spam"

### Exercise 4: Structured Data Extraction
Extract movie information (title, year, genre) from text and output as JSON.

### Exercise 5: Build a Template
Create a reusable function that translates text to any language with consistent formatting.

In [ ]:
# Your exercise code here
# Exercise 1:


# Exercise 2:


# Exercise 3:


# Exercise 4:


# Exercise 5:


## Key Takeaways

1. **System messages** set AI behavior - use them to define role and style
2. **Temperature** controls creativity: low (0-0.3) for consistency, high (0.7-2.0) for variety
3. **Few-shot learning** shows examples of desired input/output patterns
4. **Specific prompts** with constraints yield better results than vague requests
5. **Structured outputs** (JSON, tables) make AI responses easier to process programmatically

## Next Steps

In Lesson 3, we'll explore:
- Streaming responses for real-time output
- Building interactive chat interfaces
- Handling partial responses and errors

## Additional Resources

- [OpenAI Prompt Engineering Guide](https://platform.openai.com/docs/guides/prompt-engineering)
- [Best Practices for Prompt Engineering](https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api)